In [ ]:
%matplotlib inline

In [ ]:
import time
import os
import sys
from math import gcd
from collections import defaultdict, OrderedDict
import json

import xml.etree.ElementTree as ET
from xml.dom import minidom

import numpy as np
import pandas as pd

import openslide
from PIL import Image
from PIL import ImageDraw

py_src_code_dir = '../src/python'
sys.path.insert(0, py_src_code_dir)
from digipath_toolkit import get_sample_selection_mask, get_strided_fence_array
from digipath_toolkit import get_patch_location_array_for_image_level

data_dir = '../../DigiPath_MLTK_data'
zip_tank = '../../DigiPath_MLTK_data/zipTank/wsi_annotation_sample/'
xml_name = os.path.join(zip_tank, 'e39a8d60a56844d695e9579bce8f0335.xml')
c_lab_id_fn = os.path.join(zip_tank, 'class_label_id.csv')

im_dir = '../../DigiPath_MLTK_data/RegistrationDevData'
im_file = 'e39a8d60a56844d695e9579bce8f0335.tiff'
image_file_name = os.path.join(im_dir, im_file)

In [ ]:
tree = ET.parse(xml_name)
root = tree.getroot()
for region in root.iter('class_label_text'):
    print(region)


In [ ]:
tree = ET.parse(xml_name)
root = tree.getroot()
for label in root.findall('Regions/Region_Id'):
    print(type(label))

In [ ]:
with open(xml_name, 'r') as fh:
    lines = fh.read()

dom = minidom.parseString(lines)
regions = dom.getElementsByTagName('Region')
print(type(regions), len(regions))
for r in regions:
    print(r.getAttribute('Text'), r.getAttribute('Id'), r.getAttribute('Type'), r.getAttribute('GeoShape'), '\n')

```python
from xml.dom import minidom

xml_obj = minidom.parse(xml_file_name)
regions_dom = xml_obj.getElementsByTagName("Region")

regions_list = []
for reg_dom in regions_dom:
    tmp_dict = {}
    vertices = reg_dom.getElementsByTagName("Vertex")
    tmp_dict['region_Id'] = reg_dom.getAttribute('Id')
    tmp_dict['class_label_text'] = reg_dom.getAttribute('Text')
    tmp_dict['class_label_Id'] = reg_dom.getAttribute('Type')
    tmp_dict['region_geo_shape'] = reg_dom.getAttribute('GeoShape')
    tmp_dict['coords'] = np.zeros((len(vertices), 2))
    for i, vertex in enumerate(vertices):
        tmp_dict['coords'][i][0] = vertex.attributes['X'].value
        tmp_dict['coords'][i][1] = vertex.attributes['Y'].value

    regions_list.append(tmp_dict)
```

In [ ]:
help(minidom)


# Annotation: labeled patches from .xml, .csv, .svs
(and display all for developer sanity check)

## Convert xml & csv input to a region_id dict

## Create heirarchical mask set from the region_id dict

## Write labeled patches from the heirarchical mask set

```text
Questions:
    We should still create a threshold mask as before to omit any blank areas included in an annotation?
    Yes.
 
    When an entire label is eclipsed by a higher priority label (Empty) should the function issue a warning?
    No.
 
    The TFRecord could now have more than one “class_label” and,
    the “label” field that TFRecord owns is still just a sequence number… 
    thus it may be more difficult to select training/test subsets.
    Ergo: should each “class_label” aka “class_label_text” be in a different TFRecord file?
    No - one file is fine
```

## Givin a WSI, an Annotation File and an Annotation Labels priority dictionary file

1) Annotation file must follow the QuPath Annotation convention. <br>
2) Labels dictionary required to assign priority. <br>
****
```python
# read the Annotation File and the labels priority dict into a regions dictionary:
# read the regions dictionary into a labels dict
#    for each label
#        get the mask,
#        add the patches to the tfrecord or write the files,

```

### Issue: thumbnail_divisor too large makes patch test smaller than one pixel.

In [ ]:
"""
from math import gcd
from collections import defaultdict
import json

from xml.dom import minidom

import openslide

from PIL import Image
from PIL import ImageDraw
"""

# def get_nearest_thumbnail_divisor(patch_dim, thumbnail_divisor):
#     """ nearest_thumbnail_divisor = get_nearest_thumbnail_divisor(patch_dim, thumbnail_divisor)
    
#     Args:
#         patch_dim:                  integer size of patch (assuming square patch)
#         thumbnail_divisor:          integer thumbnail_divisor
        
#     Returns:
#         nearest_thumbnail_divisor:  largest, nearest in list of divisors of patch_dim
#     """
#     patch_size = int(patch_dim)
#     nearest_thumbnail_divisor = thumbnail_divisor
#     even_divisors_list = get_even_thumbnail_divisors(patch_dim)
    
#     min_diff = patch_size
#     nearest_div = 1
#     for t in even_divisors_list:
#         d = t - thumbnail_divisor
#         da = np.abs(d)
#         if min_diff >= da:
#             min_diff = da
#             nearest_thumbnail_divisor = t

#     return nearest_thumbnail_divisor

# def get_even_thumbnail_divisors(patch_dim):
#     """ Usage: even_divisors_list = get_even_thumbnail_divisors(patch_size)
#     find thumbnail divisors that won't distort the patch size | stride in the thumbnail image
    
#     Args:
#         patch_dim:              integer size of patch (assuming square patch)
        
#     Returns:
#         even_divisors_list:     unique factors of patch_dim
#     """
#     patch_size = int(patch_dim)
#     thum_div_set = {1}
#     for trial_div in range(2, 1 + patch_size // 2):
#         thum_div_set.add(gcd(patch_size, trial_div))
        
#     return sorted(list(thum_div_set))

def get_labels_dict(class_labels_id_file_name):
    """ labels_dict = get_labels_dict(class_labels_id) 
    Args:
        class_labels_id_file_name:  .csv file with columns [Label, ID, Priority]
    Returns:
        labels_dict:                python dict of dicts:
                                        {label_id_(n): {ID: x, Priority: y}, ...}
    """
    # allocate dict of dicts
    labels_dict = defaultdict(dict)
    
    # read the file
    lines = ''
    try:        
        with open(class_labels_id_file_name, 'r') as fh:
            lines = fh.readlines()
    except:
        print('failed opening: ', class_labels_id_file_name)
        lines = ''
        pass
    
    # read the lines into the dict
    if len(lines) > 0:
        for line in lines:
            line_list = line.strip().split(',')
            if len(line_list) > 1 and line_list[0] != 'Label':
                labels_dict[line_list[0]] = {'ID': line_list[1], 'Priority': line_list[1]}
                
    return labels_dict


def get_xml_list_of_dicts(xml_file_name):
    """ regions_list = get_xml_list_of_dicts(xml_file_name) 
    Args:
        xml_file_name:  required TagName
                            Vertex
                        required Attributes
                            Id
                            Text
                            Type
                            GeoShape
    Returns:
        regions_list:   list of dicts with keys:
                            region_Id
                            class_label_text
                            class_label_Id
                            region_geo_shape
                            coords
    """
    
    xml_obj = minidom.parse(xml_file_name)
    regions_dom = xml_obj.getElementsByTagName("Region")

    regions_list = []
    for reg_dom in regions_dom:
        tmp_dict = {}
        vertices = reg_dom.getElementsByTagName("Vertex")
        tmp_dict['region_Id'] = reg_dom.getAttribute('Id')
        tmp_dict['class_label_text'] = reg_dom.getAttribute('Text')
        tmp_dict['class_label_Id'] = reg_dom.getAttribute('Type')
        tmp_dict['coords'] = np.zeros((len(vertices), 2))
        for i, vertex in enumerate(vertices):
            tmp_dict['coords'][i][0] = vertex.attributes['X'].value
            tmp_dict['coords'][i][1] = vertex.attributes['Y'].value
            
        regions_list.append(tmp_dict)
        
    return regions_list


def get_region_Id_dict(xml_file_name, class_labels_id_file_name):
    """ region_id_dict = get_region_Id_dict(xml_file_name, class_labels_id_file_name) 
    """
    # read the xml file into a list of dicts
    regions_list = get_xml_list_of_dicts(xml_file_name)
    
    # read the csv file into a dict of dicts Label: {ID: x, Priority: y}
    label_dict = get_labels_dict(class_labels_id_file_name)
    
    # initialize the output dictionary
    region_id_dict = defaultdict(dict)
    
    # enter each region into the output dict
    for region_dict in regions_list:
        # extract and cast the region Priority from the csv file
        region_priority = int(label_dict[region_dict['class_label_text']]['Priority'])
        
        # construct the rest of the region dict from the xml file
        ridic = {'class_label_text': region_dict['class_label_text'], 
                 'class_label_Id': region_dict['class_label_Id'], 
                 'Priority': region_priority, 
                 'coords': region_dict['coords']}
        
        region_id_dict[int(region_dict['region_Id'])] = ridic
        
    return region_id_dict

    
def regions_dict_to_priority_dict(regions_dict):
    """ priority_dict, priority_list =   regions_dict_to_priority_dict(regions_dict) 
                        convert a regions_id_dict to a labels dict with data preserved
    
    Args:
        regions_dict:   such as returned by get_region_Id_dict(xml_file, csv_labels)
        
    Returns:
        priority_dict:    same data with labels as keys to list of regions dicts
        priority_list:    ordered highest to lowest (largest)
    """
    priority_list = []
    priority_dict = defaultdict(list)
    for k_reg_id, reg_dict in regions_dict.items():
        reg_dict['region_Id'] = k_reg_id
        priority_dict[reg_dict['Priority']].append(reg_dict)
        priority_list.append(reg_dict['Priority'])
        
    priority_list = sorted(priority_list, reverse=True)
        
    return priority_dict, priority_list


def get_region_mask(region_coords, thumbnail_divisor, thumbnail_size): # image_dimensions):
    """ mask_im, img = get_region_mask(region_coords, thumbnail_divisor,image_dimensions) 
    """
    # scale the region coords tuple with the thumbnail_divisor as type int
    xy_list = (region_coords / thumbnail_divisor).astype(np.int).tolist()
    xy_list = [(p[0], p[1]) for p in xy_list ]
    
    img = Image.fromarray(np.zeros(thumbnail_size).astype(np.uint8))
    
    # make it a Pillow Draw and draw the polygon from the list of (x,y) tuples
    draw = ImageDraw.Draw(img)
    draw.polygon(xy_list, fill="white")
    
    # create the logical mask for patch selection in the return variable
    return np.array(img) > 0


def get_select_bounds_from_mask(mask_mat, xy='x'):
    """ Usage: start_stop_dict = get_select_bounds_from_mask(mask_mat, xy='x')
    find the first and last unmasked row (y) or col (x) in the mask image input mask_mat
    
    Args:
        mask_mat:           2d numpy binary array
        xy:                 character x for x axis or y for y axis
        
    Returns:
        start_stop_dict:    {xy+'_start': _start_, xy+'_end': _stop_}
        
    """
    # initialize
    _start_ = None
    _stop_ = None
    
    # translate input variables
    if xy == 'x':
        axis = 1
        
    elif xy == 'y':
        axis = 0
        
    # sum of axis: sum_of_rows is x, axis=1,
    sum_of_axis = mask_mat.sum(axis=axis)
    current_greater_than = 0
    for k in range(sum_of_axis.size):
        if sum_of_axis[k] > 0:
            current_greater_than = k
            if _start_ is None:
                _start_ = k
    
    # set the last row if a first row one more were found to contain ones
    if not _start_ is None and current_greater_than > _start_:
        _stop_ = current_greater_than
    
    # cover the all the way to the include all cases
    if _start_ is None:
        _start_ = 0
        
    if _stop_ is None:
        _stop_ = k
        
    # name the return values with the expected x, y input
    start_stop_dict = {xy+'_start': _start_, xy+'_end': _stop_}
    
    return start_stop_dict

def get_labeled_mask_dict(run_parameters):
    """ Usage: labeled_masks_dict = get_labeled_masks_dict(run_parameters)
    Prioritized dictionary for each region defined in the input xml and csv
    
    {label: patch_selection_mask, ... } 
    
    Args:
        run_parameters:             with keys:
                                        csv_file_name
                                        xml_file_name
                                        thumbnail_divisor
                                        wsi_filename
                                    
    Returns:
        heirarchical_mask_dict:     {label_1: mask_image, ...}
        
    """  
    # define the return variable
    labeled_masks_dict = defaultdict(dict)
    
    # assign local names
    wsi_filename = run_parameters['wsi_filename']
    csv_file_name = run_parameters['csv_file_name']
    xml_file_name = run_parameters['xml_file_name']
    patch_select_method = run_parameters['patch_select_method']
    
    # Stride will not scale unless thumbnail_divisor is made of factors of patch_height & patch_width
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    
    patch_dim = max(run_parameters['patch_height'], run_parameters['patch_width'])
    #thumbnail_divisor = get_nearest_thumbnail_divisor(patch_dim, thumbnail_divisor)
    #print('Using thumbnail_divisor =', thumbnail_divisor)
    
    
    patch_height = max(1, run_parameters['patch_height'] // thumbnail_divisor)
    patch_width = max(1, run_parameters['patch_width'] // thumbnail_divisor)
    
    if 'patch_stride_fraction' in run_parameters:
        patch_stride = run_parameters['patch_stride_fraction']
    else:
        patch_stride = 1.0
    
    # get the priority dict and list of what is in it
    regions_dict = get_region_Id_dict(xml_file_name, csv_file_name)
    priority_dict, priority_list = regions_dict_to_priority_dict(regions_dict)
    
    #       initialize the priority mask
    os_im_obj = openslide.OpenSlide(wsi_filename)
    image_dimensions = os_im_obj.dimensions
    thumbnail_size = (image_dimensions[0] // thumbnail_divisor, image_dimensions[1] // thumbnail_divisor)
    small_im = os_im_obj.get_thumbnail(thumbnail_size)
    os_im_obj.close()

    higher_priorities_mask = get_sample_selection_mask(small_im, patch_select_method, run_parameters=None)
    thumbnail_size = higher_priorities_mask.shape
    
    # iterate the priority dict into the heirarchical mask set 
    # -- subtracting all higher priority masks from the new lower ones
    for p in priority_list:
        if len(priority_dict[p]) > 1:
            print('\n\n\t\tDire Warning: In Function get_labeled_mask_dict\n\n')
            print('\n\n\t\tDire Warning: More than one label with same Priority\n\n')
            print('\n\n\t\tDire Warning: Only using first label in each Priority Level\n\n')
        p_dict = priority_dict[p]    
        #p_dict = priority_dict[p][0]
        label = p_dict['class_label_text']
        this_mask = get_region_mask(p_dict['coords'], thumbnail_divisor, thumbnail_size) # image_dimensions)
        this_mask = np.logical_and(np.logical_not(higher_priorities_mask), this_mask)
        
        if this_mask.sum() > 0:
            higher_priorities_mask = np.logical_or(this_mask, higher_priorities_mask)
            
            #                              May not need this in the return -- 
            p_dict['mask_im'] = this_mask
            
            start_stop_rows = get_select_bounds_from_mask(this_mask, xy='y')
            row_start, row_end = start_stop_rows['y_start'], start_stop_rows['y_end']
            rows_fence_array = get_strided_fence_array(patch_height, patch_stride, row_start, row_end)
            p_dict['rows_fence_array'] = rows_fence_array[:,0] * thumbnail_divisor

            start_stop_cols = get_select_bounds_from_mask(this_mask, xy='x')
            col_start, col_end = start_stop_cols['x_start'], start_stop_cols['x_end']
            cols_fence_array = get_strided_fence_array(patch_width, patch_stride, col_start, col_end)
            p_dict['cols_fence_array'] = cols_fence_array[:,0] * thumbnail_divisor
            
            labeled_masks_dict[p] = p_dict
            
    return labeled_masks_dict


In [ ]:
def get_priority_dict(xml_file_name, class_labels_id_file_name):
    """ priority_dict = get_region_Id_dict(xml_file_name, class_labels_id_file_name) 
    """
    # read the xml file into a list of dicts
    regions_list = get_xml_list_of_dicts(xml_file_name)
    
    # read the csv file into a dict of dicts Label: {ID: x, Priority: y}
    label_dict = get_labels_dict(class_labels_id_file_name)
    
    # initialize the output dictionary
    region_id_dict = defaultdict(dict)
    
    # enter each region into the output dict
    for region_dict in regions_list:
        # extract and cast the region Priority from the csv file
        region_priority = int(label_dict[region_dict['class_label_text']]['Priority'])
        
        # construct the rest of the region dict from the xml file
        ridic = {'class_label_text': region_dict['class_label_text'], 
                 'class_label_Id': region_dict['class_label_Id'], 
                 'Priority': region_priority, 
                 'coords': region_dict['coords']}
        
        region_id_dict[int(region_dict['region_Id'])] = ridic
        
    return region_id_dict    
    
p_dict = get_priority_dict(xml_file_name=xml_name, class_labels_id_file_name=c_lab_id_fn)


In [ ]:
labels_df = pd.read_csv(c_lab_id_fn).fillna('null')
labels_df

In [ ]:

Priority = labels_df.loc[labels_df['ID'] == 1]['Priority']
Label = labels_df.loc[labels_df['ID'] == 1]['Label']

print(Priority, Label)

In [ ]:
type(labels_df.loc[labels_df['ID'] == 1])
labels_df.loc[labels_df['ID'] == 1]

In [ ]:
labels_df = pd.read_csv(c_lab_id_fn, index_col=0)
od_list = []
for r, row in labels_df.iterrows():
    if isinstance(r, float) and np.isnan(r) == True:
        print('%i, %i, %s'%(row['ID'], row['Priority'], 'null'))
        od_list.append((row['Priority'], 'null'))
    else:
        print('%i, %i, %s'%(row['ID'], row['Priority'], r))
        od_list.append((row['Priority'], r))

# od_dakine = OrderedDict(sorted(od_list))
# for k, v in od_dakine.items():
#     print(k, v)

# Yo lo necissito:
```python

priority_dict = OrderedDict((priority_integer_max, {'class_label_text': clt, 'coords': np2d_array}),
                            (priority_integer_max-1, {'class_label_text': clt, 'coords': np2d_array}), ...)
```

## y no mas

In [ ]:
priority_dict_list = []

print('with file: %s'%(xml_name))
labels_df = pd.read_csv(c_lab_id_fn, index_col=0).

xml_obj = minidom.parse(xml_name)
regions_dom = xml_obj.getElementsByTagName("Region")

regions_list = []
for reg_dom in regions_dom:
    tmp_dict = {}
    vertices = reg_dom.getElementsByTagName("Vertex")
    tmp_dict['region_Id'] = reg_dom.getAttribute('Id')
    tmp_dict['class_label_text'] = reg_dom.getAttribute('Text')
    tmp_dict['class_label_Id'] = reg_dom.getAttribute('Type')
    tmp_dict['coords'] = np.zeros((len(vertices), 2))
    for i, vertex in enumerate(vertices):
        tmp_dict['coords'][i][0] = vertex.attributes['X'].value
        tmp_dict['coords'][i][1] = vertex.attributes['Y'].value

    regions_list.append(tmp_dict)
    priority = 
    priority_dict_list.append((reg_dom.getAttribute('Text'), {'class_label_text': reg_dom.getAttribute('Text')}))

In [ ]:
for k, v in get_labels_dict(class_labels_id_file_name=c_lab_id_fn).items():
    print(k, v)

In [ ]:
for r_dict in get_xml_list_of_dicts(xml_file_name=xml_name):
    for k, v in r_dict.items():
        print(k, v )
    print('\n\n')

In [ ]:
rid_dic = get_region_Id_dict(xml_file_name=xml_name, class_labels_id_file_name=c_lab_id_fn)
# priority_dict, priority_list = regions_dict_to_labels_dict(rid_dic)
priority_dict, priority_list =   regions_dict_to_priority_dict(rid_dic)
for k in priority_list:
    print('priority level:', k, 'number of items', len(priority_dict[k]), 'type', type(priority_dict[k][0]))
    for lbl, v in priority_dict[k][0].items():
        if isinstance(v, np.ndarray):
            print('\t', lbl, v.shape)
        else:
            print('\t', lbl, v)
    print('\n')

In [ ]:
data_dir = '../../DigiPath_MLTK_data'
output_dir = '../../DigiPath_MLTK_data/annotation_test/results'
if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)

wsi_file = 'RegistrationDevData/e39a8d60a56844d695e9579bce8f0335.tiff'
wsi_file = os.path.join(data_dir, wsi_file)
csv_file = 'wsi_annotation_sample/class_label_id.csv'
csv_file = os.path.join(data_dir, csv_file)
xml_file = 'wsi_annotation_sample/e39a8d60a56844d695e9579bce8f0335.xml'
xml_file = os.path.join(data_dir, xml_file)

run_parameters = {'method': 'annotations_to_dir', 
                  'output_dir': output_dir,
                  'wsi_filename': wsi_file, 
                  'csv_file_name': csv_file,
                  'xml_file_name': xml_file,
                  'thumbnail_divisor': 100, 
                  'patch_stride_fraction': 1.0, 
                  'image_level': 0,  
                  'patch_height': 224, 
                  'patch_width': 224, 
                  'threshold': 0, 
                  'patch_select_method': 'threshold_rgb2lab', 
                  'rgb2lab_threshold': 80}

t0 = time.time()
label_mask_dict = get_labeled_mask_dict(run_parameters)
tt = time.time() - t0
print('run_time %0.3f initial thumbnail_divisor: %0.6f\n'%(tt, run_parameters['thumbnail_divisor']))
for label, lbl_dict in label_mask_dict.items():
    print('level', label, 'label', lbl_dict['class_label_text'], '\t', type(lbl_dict), len(lbl_dict))

```text
normal <class 'numpy.ndarray'>
ink <class 'numpy.ndarray'>
offset <class 'numpy.ndarray'>
malignant <class 'numpy.ndarray'>
Region <class 'numpy.ndarray'>
```

In [ ]:
for k in lbl_dict.keys():
    print(k)

In [ ]:
print(type(lbl_dict))
rows_fence_arr = lbl_dict['rows_fence_array']
print(type(rows_fence_arr), rows_fence_arr.shape)
for idx in range(rows_fence_arr.shape[0] - 1):
    print('row: %6i\tstride: %i'%(rows_fence_arr[idx], rows_fence_arr[idx+1] - rows_fence_arr[idx]))

# Review-test notebook implemented functions:

In [ ]:
"""
        Test function: get_labeled_masks(run_parameters)
        
        Expected - one labeled mask per priority level unless mask is blank
        
"""
t0 = time.time()
# labeled_masks_dict = get_labeled_masks(run_parameters)
labeled_masks_dict = get_labeled_mask_dict(run_parameters)

for merge_label, merged_lbl_msk in labeled_masks_dict.items():
    if merged_lbl_msk['mask_im'].sum() == 0:
        print(merge_label, 'Is Empty Mask')
    else:
        m_lbl_im = Image.fromarray((merged_lbl_msk['mask_im'].astype(np.uint8) * 255))
        print('mask for %s'%(merge_label))
        display(m_lbl_im)
    
print('total run time: %0.3f'%(time.time() - t0))

In [ ]:
"""
        Test - demo            regions_dict = get_region_Id_dict(xml_name, c_lab_id_fn)

        Test - demo            priority_dict, priority_list= regions_dict_to_priority_dict(regions_dict)

Note that in this test set (xml, csv) there is only one class label per priority level
"""
regions_dict = get_region_Id_dict(xml_name, c_lab_id_fn)
priority_dict, priority_list = regions_dict_to_priority_dict(regions_dict)

for p in priority_list:
    lbl_dict_list = priority_dict[p]
    print('\nPriority:', p, '\t(%i dictionaries)'%(len(lbl_dict_list)))
    for lbl_dict in lbl_dict_list:
        for k, v in lbl_dict.items():
            if isinstance(v, np.ndarray):
                print('%20s:'%(k), v[0,:])
                for v_ix in range(len(v) - 1):
                    print('%20s '%(' '), v[v_ix+1,:])
            else:
                print('%20s:'%(k), v)

In [ ]:
# thumbnail divisor is remainder madness unless it is a divisor of pathch size 
patch_size = 224
thumbnail_divisor_test = 5

thm_div_list = get_even_thumbnail_divisors(patch_dim=patch_size)
print('get_even_thumbnail_divisors returns list:\n')
for t in thm_div_list:
    d = t - thumbnail_divisor_test
    da = np.abs(d)
    print(t, da)

print('\n\nfunction returns', get_nearest_thumbnail_divisor(patch_size, thumbnail_divisor_test))

In [ ]:
"""
        n_pixels_per_stride = patch_size * patch_stride
        patch_stride = n_pixels_per_stride / patch_size
"""
_MIN_STRIDE_PIXELS = 3
_patch_stride = 0.1
_patch_width = _patch_height = 224

#       assure minimum stride s.t. arrays advance by at least MIN_STRIDE_PIXELS
_patch_stride = max(_patch_stride, (_MIN_STRIDE_PIXELS / min(_patch_width, _patch_height) ) )
print(_patch_stride, _patch_stride * min(_patch_width, _patch_height) )

In [ ]:
# from PIL import Image

im_rgbA = Image.fromarray((np.random.random((200,200,4))* 255).astype(np.uint8) )
print('image (with alpha channel) RGB')
display(im_rgbA)

im_rgb = im_rgbA.convert('RGB')
print('image (alpha channel removed) RGB')
display(im_rgb)

im_gray_from_rgb = im_rgb.convert('L')
print('image gray from RGB')
display(im_gray_from_rgb)

im_gray = Image.fromarray((np.random.random((200,200))* 255).astype(np.uint8) )
print('image grayscale from 2D array')
display(im_gray)